In [38]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN,LSTM
from keras.optimizers import Adam, SGD
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from keras.losses import MeanSquaredLogarithmicError,MeanSquaredError,MeanAbsolutePercentageError,Huber
from keras.metrics import RootMeanSquaredError
from sklearn.metrics import mean_squared_error

In [39]:
import matplotlib.pyplot as plt 
from datetime import datetime,timedelta
import numpy as np
import pandas as pd
import seaborn as sns
from keras.models import load_model

In [40]:
def pasar_a_datetime(fecha):
    #format_string = '%Y-%m-%d'
    anio,mes,dia= fecha.split('-')
    aux_dia = dia.split(' ')
    if len(aux_dia)>1:dia = aux_dia[0]
    res = datetime(int(anio),int(mes),int(dia))
    return res

In [41]:
def scale_datasets(x_train, x_test):

  """
  Standard Scale test and train data
  Z - Score normalization
  """
  standard_scaler = StandardScaler()
  x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
  )
  x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
  )
  return x_train_scaled, x_test_scaled

In [42]:
def separar_fecha(data):
    data['dia'] = ''
    data['mes'] = ''
    data['anio'] = ''
    data['dia'] = data['fecha_mediana'].map(lambda x:x.day)
    data['mes'] = data['fecha_mediana'].map(lambda x:x.month)
    data['anio'] = data['fecha_mediana'].map(lambda x:x.year)

In [43]:
def encoding_Fuente(tabla,fuentes):
    #fuentes_labels = publicaciones['fuente'].unique()
    numbers = [i+1 for i in range(len(fuentes))]
    simple_encoding = dict(zip(fuentes,numbers))
    tabla['Fuente'] = tabla['Fuente'].map(lambda x:simple_encoding[x])

In [44]:
def formato_entrenado(datos_publicaciones_input,partidos):
    datos_publicaciones = datos_publicaciones_input.copy()
    datos_publicaciones['fecha']=datos_publicaciones['fecha'].map(lambda x:pasar_a_datetime(x))
    fuentes = datos_publicaciones['fuente'].unique()
    #partidos = datos_publicaciones['Partido'].unique()
    #datos_publicaciones['fecha'] = datos_publicaciones['fecha'].map(lambda x:pasar_a_datetime(x))

    #datos_publicaciones['cantidad_likes'].quantile(0.95)
    datos_publicaciones = datos_publicaciones[datos_publicaciones['cantidad_likes']<25000]

    
    agrupado_likes = datos_publicaciones.groupby(['Partido','fuente'])['cantidad_likes'].median()
    fecha_median = datos_publicaciones['fecha'].median()

    df = pd.DataFrame()

    for p in list(partidos):

        if p in agrupado_likes.index:
            likes_partidos = agrupado_likes[p]
            fuentes_partido = likes_partidos.index
            likes_median = []

            for c in fuentes:
                if c in fuentes_partido:
                    likes_median.append(likes_partidos[c])
                else:
                    likes_median.append(0)
        
            df[p] = likes_median
        else:
            df[p] = [0]*len(fuentes)


    df['Fuente'] = fuentes
    df['fecha_mediana'] = fecha_median

    
    separar_fecha(df)
    df.drop(['fecha_mediana','anio'],axis=1,inplace=True)
    encoding_Fuente(df,fuentes)
    return df

In [45]:
def acomodar_prediccion(prediccion,datos_formateado):
    partidos = datos_formateado.columns[:5]

    resultados_mean = []
    for i in range(5):
        resultados_mean.append(prediccion[:,i].mean())

    total = sum(resultados_mean)
    resultados_norm = []
    for r in resultados_mean:
        aux = 100*r/total
        resultados_norm.append(aux)

    resultados_norm = [round(x,1) for x in resultados_norm]

    df = pd.DataFrame({'Partido':partidos,'Resultado':resultados_norm})

    return df

In [46]:
def ordenar_fecha(data):
    data['fecha_mediana'] = data['fecha_mediana'].map(lambda x: pasar_a_datetime(x))
    data.sort_values(by='fecha_mediana',inplace=True)

In [47]:
def preparar_datos_entrenamiento(X_train, X_test):
    x_train_scaled, x_test_scaled,stdscaler = scale_datasets(X_train, X_test)
    return x_train_scaled, x_test_scaled,stdscaler 

In [48]:
def modelo_arquitectura(in_dim,out_dim):
    learning_rate = 0.0005
    
    metrica = RootMeanSquaredError()
    perdida = MeanSquaredError()

    model = Sequential()
    model.add(Dense(units=in_dim,input_dim=in_dim,activation='softplus'))
    model.add(Dense(units=(in_dim+out_dim)//2,activation='softplus'))
    model.add(Dense(units=out_dim, activation='softplus')) 
    model.compile(loss=perdida, 
                optimizer=SGD(learning_rate=learning_rate,
                              momentum=0.8), 
                metrics=[metrica]) 
    
    return model

In [49]:
def seleccionar_datos(datos):
    cols = datos.columns.to_list()
    idx = cols.index('Fuente')
    x_cols = cols[:idx+1]+cols[-2:]
    y_cols = cols[idx+2:-2]

    x = datos.loc[:,x_cols]
    y = datos.loc[:,y_cols]
    return x,y

# Idea

Dada una fecha F,
Selecionar datos del ajuste polinomial con 'fecha_mediana'<F

Entrenar el modelo con esos datos y predecir para la fecha F
Con los datos de las Publicaciones (Con una semana de ventana)

Reportar MSE como puntaje para cada fecha en 'fecha_mediana'

In [50]:
data_encuestas_poly = pd.read_csv('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/modeling/Preparando_datos/data_encuestas_poly.csv')
publicaciones = pd.read_csv('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/modeling/Preparando_datos/publicaciones_politicas.csv')
Encuestas_primera_vuelta = pd.read_csv('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/getting_data/encuestas/Encuestas_primera_vuelta.csv')


In [51]:
fechas_encuestas = data_encuestas_poly['fecha_mediana'].unique()

In [52]:
def filtrar_data_fecha(datos, fecha):
    res = datos[datos['fecha_mediana']<fecha]
    return res

In [53]:
def entrenar_modelo(datos):
    ciclos = 300
    x,y = seleccionar_datos(datos)
    model = modelo_arquitectura(8, 5)
    std_scaler = StandardScaler()

    x_scaled = std_scaler.fit_transform(x)
    model.fit(x_scaled,y, epochs=ciclos, verbose=0,batch_size=16)
    return model,std_scaler


In [54]:
def filtrar_fechas(datos_publicaciones, inicio, final):
    aux_datos_publicaciones = datos_publicaciones['fecha'].map(lambda x:pasar_a_datetime(x))
    res = datos_publicaciones[(aux_datos_publicaciones>inicio) & (aux_datos_publicaciones<final)]
    return res

In [55]:
def evaluar_modelo(model,fecha,datos_publicaciones,scaler):

    fecha_final = datetime(2023,10,20)
    semana = timedelta(days=14)
    fecha_semana = fecha_final - semana
    partidos = datos_publicaciones['Partido'].unique()
    semana_1 = filtrar_fechas(datos_publicaciones,fecha_semana,fecha_final)
    semana_1_format = formato_entrenado(semana_1,partidos)
    semana_1_f_scaled = scaler.fit_transform(semana_1_format)
    prediccion = model.predict(semana_1_f_scaled,verbose=0)

    res = acomodar_prediccion(prediccion,semana_1_format)
    return res

    

In [56]:
def calcularScore(df):
    aux_encuestas = Encuestas_primera_vuelta.iloc[0,[2,0,1,3,4]].reset_index()
    aux_encuestas.rename(columns=dict(zip(aux_encuestas.columns,['Partido','Esperado'])),inplace=True)
    comparando = pd.concat([df,aux_encuestas],axis=1).dropna()
    A = comparando['Resultado']
    B = comparando['Esperado']
    mse = -mean_squared_error(B,A)
    return mse

In [57]:
def entrenando_por_fechas(fechas):
    dfs = []
    fechas_errores = []
    for k,f in enumerate(fechas):
        datos_fecha = filtrar_data_fecha(data_encuestas_poly,f)
        if len(datos_fecha)>0:
            print(100*(k+1)/len(fechas))
            model,scaler = entrenar_modelo(datos_fecha)
            df = evaluar_modelo(model,f,publicaciones,scaler)
            dfs.append(df)
        else: fechas_errores.append(f)

    return dfs
    res = []
    for df in dfs:
        res.append(calcularScore(df))
    return res,fechas_errores


In [58]:
df = entrenando_por_fechas(fechas_encuestas[5:6])

100.0


In [59]:
df[0]

,Partido,Resultado
0,La Libertad Avanza,32.4
1,Union por la Patria,38.3
2,Juntos por el Cambio,23.6
3,Frente de Izquierda y Trabajadores,2.6
4,Hacemos juntos nuestro Pais,3.1


In [37]:
df[0]

,Partido,Resultado
0,La Libertad Avanza,32.4
1,Union por la Patria,38.3
2,Juntos por el Cambio,23.6
3,Frente de Izquierda y Trabajadores,2.6
4,Hacemos juntos nuestro Pais,3.1


In [22]:
data_scoring_fecha = pd.DataFrame()
data_scoring_fecha['Fechas'] = fechas_encuestas[1:]
data_scoring_fecha['Score'] = scores

In [23]:
data_scoring_fecha.to_csv('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/dashboard/data_scoring_fecha_1.csv',index=False)

In [26]:
data = pd.read_csv('C:/Users/54911/OneDrive/Escritorio/Data Science/Elecciones + IA/dashboard/data_scoring_fecha.csv')